In [1]:
from pandas import read_csv
from matplotlib import pyplot
import sys
import numpy as np
import pandas as pd
from datetime import datetime
from keras.utils import to_categorical


2024-12-21 19:30:13.806603: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-21 19:30:13.815409: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734805813.823445  122815 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734805813.825857  122815 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-21 19:30:13.834791: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
colors = ['#BB0000', '#0000BB', 'green','red', 'yellow', 'pink']

def prinUserSensorData(dataS, label,s):
    df = pd.DataFrame()
    ts=[]
    ls=[]
    #print(dataS.shape)
    for t in range(0,dataS.shape[0]):
        ts.append(dataS[t][0])
    df['Time']=ts
    #print(ts)
#    for s in range(1,dataS.shape[1]):
    ls.append(label+" #column-"+str(s))
    #print(dataS[:,s].shape)
    df[label+" #column-"+str(s)]=dataS[:,s]

    print(ls)
    
    df.plot(x="Time",y=ls,subplots=True, legend=True, figsize=(15,5), color=colors[s-1])    
    


In [3]:
import numpy as np
import sys

folder = "./data"

users = ["Mix"]
files = ["der1", "der2",
         "der3", "der4",
         "der5", "der6",
         "der7", "der8",
         "izq1", "izq2",
         "izq5", "izq6",
         "izq7", "izq8",
         "sim1", "sim2",
         "sim3", "sim4",
         "sim5", "sim6",
         "sim7", "sim8"]

cross = [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]


sensors = ["s1", "s2"]

tW = list(range(-5000, 100, 50))

print(tW)
print(len(tW))

tStep = 150

index_t = 0

def getAVG(vs):
    if len(vs) == 0:
        return 0
    if np.isnan(np.median(vs)):
        return -1
    return np.median(vs)

def smoothData(data):
    for ix in range(1, 14):
        data[:, ix] = data[:, ix].astype(float)
        lastV = 0.0
        for jx in range(len(data[:, ix])):
            data[jx, ix] = data[jx, ix] * 0.2 + lastV * 0.8
            lastV = data[jx, ix]

def smoothDataTime(data):
    data[:, 0] = data[:, 0].astype(float)
    tA = []
    lastT = -1
    for jx in range(len(data[:, 0])):
        t0 = data[jx, 0]
        if t0 > lastT + 50:
            lastT = t0
            tA.append(t0)
    retV = np.zeros((len(tA), 14))
    for ix in range(1, 14):
        data[:, ix] = data[:, ix].astype(float)
        lastT = -1
        lV = []
        vA = []
        for jx in range(len(data[:, ix])):
            t0 = data[jx, 0]
            if t0 > lastT + 50:
                lastT = t0
                vA.append(np.median(lV))
                lV = []
            else:
                lV.append(data[jx, ix])
        retV[:, ix] = np.array(vA)
    retV[:, 0] = np.array(tA)
    return retV

def cleanDataFile(filename):
    ret = []
    count = 0
    with open(filename) as file:
        next(file) 
        for line0 in file:
            count += 1
            line = line0.strip()
            line = line.replace(',', '.').split('\t')  
            if len(line) != 14:
                print("NOT 14# in ", count, " -> ", line)
                continue
            try:
                # Convert each item in the line to float; replace empty strings with 0.0
                ret.append([float(x) if x else 0.0 for x in line])
            except ValueError as e:
                print("Error in line ", line, ":", e)
    return ret


def getLabels(t0, labels):
    for l in labels:
        if l[0] <= t0 <= l[1]:
            return l[2]
    return None

[-5000, -4950, -4900, -4850, -4800, -4750, -4700, -4650, -4600, -4550, -4500, -4450, -4400, -4350, -4300, -4250, -4200, -4150, -4100, -4050, -4000, -3950, -3900, -3850, -3800, -3750, -3700, -3650, -3600, -3550, -3500, -3450, -3400, -3350, -3300, -3250, -3200, -3150, -3100, -3050, -3000, -2950, -2900, -2850, -2800, -2750, -2700, -2650, -2600, -2550, -2500, -2450, -2400, -2350, -2300, -2250, -2200, -2150, -2100, -2050, -2000, -1950, -1900, -1850, -1800, -1750, -1700, -1650, -1600, -1550, -1500, -1450, -1400, -1350, -1300, -1250, -1200, -1150, -1100, -1050, -1000, -950, -900, -850, -800, -750, -700, -650, -600, -550, -500, -450, -400, -350, -300, -250, -200, -150, -100, -50, 0, 50]
102


In [ ]:
X=[]
Y=[]
S=[]
N=0



for ixu,us in enumerate(users):    
    for ixc,cl in enumerate(files):
            t0=0
            tMaxMin=sys.maxsize 
            interval=False
            #idx_sensor={}
            sensorData={}
            for isx,sn in enumerate(sensors):
                #idx_sensor[sn]=[]
                print(folder+"/"+us+"/"+cl+"/"+str(sn)+".txt")
                data=cleanDataFile(folder+"/"+us+"/"+cl+"/"+str(sn)+".txt")
                data=np.array(data)
                smoothData(data)
                #prinUserSensorData(data)
                sensorData[sn]=data
                tMax=0
                tMin=sys.maxsize 
                for ixd,d in enumerate(data):
                    #idx_sensor[sn].append(ixd)
                    if(d[index_t]>tMax):
                        tMax=d[index_t]
                    if(d[index_t]<tMin):
                        tMin=d[index_t]                        
                tMaxMin=min(tMaxMin,tMax-tMin)
                
            labels=read_csv(folder+"/"+us+"/"+cl+"/info.txt", sep="\t",parse_dates=True,header=None, decimal=".")
            labels.iloc[:, 0] = labels.iloc[:, 0].astype(int)  # Convertir la primera columna a entero
            labels.iloc[:, 1] = labels.iloc[:, 1].astype(int)  # Convertir la segunda columna a entero


            labels=labels.values
            print(labels)
            
            tMaxMin=int(tMaxMin)
            print(cl," tMaxMin:",tMaxMin)
                        
            for t in range(0-min(tW),tMaxMin-max(tW),tStep):
                print(t/(tMaxMin-max(tW))*100, "t#",t)
                
                label=getLabels(t,labels)
                if (not label):
                    continue
                xi=[]
                for sn in sensors:
                    data=sensorData[sn]
                    tRef=np.min(data[:,index_t])

                    
                    xi1=[]
                    xi2=[]
                    xi3=[]
                    xi4=[]
                    xi5=[]
                    xi6=[]
                    xi7=[]
                    xi8=[]
                    xi9=[]
                    xi10=[]
                    xi11=[]
                    xi12=[]
                    
                    for it in range(len(tW)-2,-1,-1):

                        tr=t+tRef+np.random.normal(0, 25)

                        t0=tr+tW[it]
                        tN=tr+tW[it+1]
                        ag1=data[np.where((data[:,index_t] >= t0) & (data[:,index_t] <=  tN))]
                        #print("\t t0,tN,#,t,it,tRef",t0,tN,len(ag1),t,it,tW[it],tRef)
                        #print("cl",cl,"sc",sc,"sn",sn,"t+:",t,"tRef:",tRef,"t0:",t0,"tN:",tN," ag1:", ag1)
                        #ag0=data[np.where((data[:,index_t] >= tr-5000) & (data[:,index_t] <=  tr+5000))]
                        
                        

                        #añadimos info sensores inerciales
                        xi1.append(getAVG(ag1[:,2].astype(float)))
                        xi2.append(getAVG(ag1[:,3].astype(float)))
                        #xi3.append(getAVG(ag1[:,4].astype(float)))
                        xi4.append(getAVG(ag1[:,5].astype(float)))
                        xi5.append(getAVG(ag1[:,6].astype(float)))
                        xi6.append(getAVG(ag1[:,7].astype(float)))      
                        xi7.append(getAVG(ag1[:,8].astype(float))) 
                        xi8.append(getAVG(ag1[:,9].astype(float))) 
                        xi9.append(getAVG(ag1[:,10].astype(float))) 
                        xi10.append(getAVG(ag1[:,11].astype(float))) 
                        xi11.append(getAVG(ag1[:,12].astype(float))) 
                        xi12.append(getAVG(ag1[:,13].astype(float))) 

                    #print("\t",np.array(xi1).shape)

                    xi.append(xi4)
                    xi.append(xi5)
                    xi.append(xi6)
                    xi.append(xi7)
                    xi.append(xi8)
                    xi.append(xi9)
                    xi.append(xi10)
                    xi.append(xi11)
                    xi.append(xi12)


                    #print("\t",np.array(xi1).shape)
                    xi.append(xi1)
                    xi.append(xi2)
                    #xi.append(xi3)
                


                #print("",np.array(xi).shape)    
                X.append(np.transpose(xi))
                Y.append(label)
                print(len(cross))
                S.append(cross[ixc])

        
X=np.array(X)
S=np.array(S)
Y=np.array(Y)

print(X.shape)
print(Y.shape)    
print(S.shape) 

                

./data/Mix/der1/s1.txt
./data/Mix/der1/s2.txt
[[40180 64570 'der']
 [79280 100050 'der']]
der1  tMaxMin: 151810
3.2946758039008963 t# 5000
3.393516078017923 t# 5150
3.4923563521349497 t# 5300
3.5911966262519766 t# 5450
3.6900369003690034 t# 5600
3.788877174486031 t# 5750
3.8877174486030577 t# 5900
3.986557722720084 t# 6050
4.085397996837111 t# 6200
4.184238270954138 t# 6350
4.283078545071165 t# 6500
4.381918819188193 t# 6650
4.480759093305219 t# 6800
4.5795993674222455 t# 6950
4.678439641539272 t# 7100
4.777279915656299 t# 7250
4.876120189773326 t# 7400
4.974960463890353 t# 7550
5.07380073800738 t# 7700
5.172641012124407 t# 7850
5.271481286241434 t# 8000
5.370321560358461 t# 8150
5.4691618344754875 t# 8300
5.568002108592514 t# 8450
5.666842382709541 t# 8600
5.765682656826568 t# 8750
5.864522930943595 t# 8900
5.963363205060622 t# 9050
6.062203479177649 t# 9200
6.161043753294676 t# 9350
6.259884027411703 t# 9500
6.3587243015287305 t# 9650
6.4575645756457565 t# 9800
6.5564048497627825 t# 

10
51.607166381240944 t# 78350
10
51.70596759320247 t# 78500
10
51.80476880516402 t# 78650
10
51.903570017125546 t# 78800
10
52.002371229087075 t# 78950
10
52.10117244104862 t# 79100
10
52.19997365301015 t# 79250
10
52.298774864971676 t# 79400
10
52.397576076933206 t# 79550
10
52.49637728889475 t# 79700
10
52.59517850085628 t# 79850
10
52.69397971281781 t# 80000
10
52.79278092477935 t# 80150
10
52.89158213674088 t# 80300
10
52.99038334870241 t# 80450
10
53.08918456066395 t# 80600
10
53.18798577262548 t# 80750
10
53.28678698458701 t# 80900
10
53.38558819654854 t# 81050
10
53.48438940851008 t# 81200
10
53.58319062047161 t# 81350
10
53.68199183243314 t# 81500
10
53.780793044394684 t# 81650
10
53.879594256356214 t# 81800
10
53.97839546831774 t# 81950
10
54.077196680279286 t# 82100
10
54.175997892240815 t# 82250
10
54.274799104202344 t# 82400
10
54.37360031616388 t# 82550
10
54.47240152812542 t# 82700
10
54.571202740086946 t# 82850
10
54.670003952048475 t# 83000
10
54.76880516401002 t# 8315

./data/Mix/sim2/s2.txt
[[108080 128260 'sim']]
sim2  tMaxMin: 151870
3.293373732051113 t# 5000
3.3921749440126465 t# 5150
3.49097615597418 t# 5300
3.5897773679357132 t# 5450
3.688578579897247 t# 5600
3.7873797918587804 t# 5750
3.886181003820314 t# 5900
3.9849822157818466 t# 6050
4.08378342774338 t# 6200
4.182584639704913 t# 6350
4.281385851666447 t# 6500
4.3801870636279805 t# 6650
4.478988275589514 t# 6800
4.577789487551048 t# 6950
4.67659069951258 t# 7100
4.775391911474114 t# 7250
4.8741931234356475 t# 7400
4.972994335397181 t# 7550
5.071795547358715 t# 7700
5.170596759320248 t# 7850
5.269397971281782 t# 8000
5.3681991832433145 t# 8150
5.467000395204848 t# 8300
5.565801607166381 t# 8450
5.664602819127914 t# 8600
5.763404031089448 t# 8750
5.8622052430509815 t# 8900
5.961006455012515 t# 9050
6.059807666974048 t# 9200
6.158608878935581 t# 9350
6.257410090897116 t# 9500
6.356211302858648 t# 9650
6.455012514820181 t# 9800
6.553813726781716 t# 9950
6.652614938743248 t# 10100
6.7514161507047

10
71.56501119747068 t# 108650
10
71.66381240943223 t# 108800
10
71.76261362139375 t# 108950
10
71.86141483335528 t# 109100
10
71.96021604531683 t# 109250
10
72.05901725727836 t# 109400
10
72.15781846923988 t# 109550
10
72.25661968120141 t# 109700
10
72.35542089316296 t# 109850
10
72.45422210512449 t# 110000
10
72.55302331708602 t# 110150
10
72.65182452904756 t# 110300
10
72.75062574100909 t# 110450
10
72.84942695297062 t# 110600
10
72.94822816493216 t# 110750
10
73.04702937689369 t# 110900
10
73.14583058885522 t# 111050
10
73.24463180081676 t# 111200
10
73.34343301277829 t# 111350
10
73.44223422473982 t# 111500
10
73.54103543670135 t# 111650
10
73.6398366486629 t# 111800
10
73.73863786062442 t# 111950
10
73.83743907258595 t# 112100
10
73.9362402845475 t# 112250
10
74.03504149650902 t# 112400
10
74.13384270847055 t# 112550
10
74.2326439204321 t# 112700
10
74.33144513239363 t# 112850
10
74.43024634435515 t# 113000
10
74.52904755631668 t# 113150
10
74.62784876827823 t# 113300
10
74.72664

./data/Mix/sim3/s2.txt
[[38690 63460 'sim']
 [84100 108080 'sim']
 [130710 152990 'sim']]
sim3  tMaxMin: 274650
1.820830298616169 t# 5000
1.8754552075746542 t# 5150
1.930080116533139 t# 5300
1.9847050254916243 t# 5450
2.0393299344501092 t# 5600
2.093954843408594 t# 5750
2.1485797523670795 t# 5900
2.2032046613255645 t# 6050
2.2578295702840494 t# 6200
2.3124544792425343 t# 6350
2.3670793882010197 t# 6500
2.421704297159505 t# 6650
2.4763292061179896 t# 6800
2.530954115076475 t# 6950
2.58557902403496 t# 7100
2.640203932993445 t# 7250
2.69482884195193 t# 7400
2.749453750910415 t# 7550
2.8040786598689005 t# 7700
2.858703568827385 t# 7850
2.9133284777858703 t# 8000
2.9679533867443557 t# 8150
3.02257829570284 t# 8300
3.0772032046613256 t# 8450
3.1318281136198105 t# 8600
3.186453022578296 t# 8750
3.241077931536781 t# 8900
3.295702840495266 t# 9050
3.3503277494537507 t# 9200
3.4049526584122356 t# 9350
3.459577567370721 t# 9500
3.5142024763292063 t# 9650
3.5688273852876913 t# 9800
3.6234522942461

...

10
33.28477785870357 t# 91400
10
33.33940276766205 t# 91550
10
33.39402767662054 t# 91700
10
33.448652585579026 t# 91850
10
33.50327749453751 t# 92000
10
33.557902403495994 t# 92150
10
33.612527312454475 t# 92300
10
33.66715222141296 t# 92450
10
33.72177713037145 t# 92600
10
33.77640203932993 t# 92750
10
33.83102694828842 t# 92900
10
33.885651857246906 t# 93050
10
33.94027676620539 t# 93200
10
33.994901675163874 t# 93350
10
34.04952658412236 t# 93500
10
34.10415149308085 t# 93650
10
34.15877640203933 t# 93800
10
34.21340131099782 t# 93950
10
34.2680262199563 t# 94100
10
34.322651128914785 t# 94250
10
34.37727603787327 t# 94400
10
34.43190094683175 t# 94550
10
34.48652585579024 t# 94700
10
34.54115076474872 t# 94850
10
34.595775673707216 t# 95000
10
34.6504005826657 t# 95150
10
34.705025491624184 t# 95300
10
34.759650400582665 t# 95450
10
34.814275309541145 t# 95600
10
34.86890021849964 t# 95750
10
34.92352512745812 t# 95900
10
34.97815003641661 t# 96050
10
35.03277494537509 t# 96200
10

10
48.79825200291333 t# 134000
10
48.85287691187181 t# 134150
10
48.9075018208303 t# 134300
10
48.962126729788785 t# 134450
10
49.016751638747266 t# 134600
10
49.07137654770575 t# 134750
10
49.126001456664234 t# 134900
10
49.18062636562273 t# 135050
10
49.23525127458121 t# 135200
10
49.2898761835397 t# 135350
10
49.34450109249818 t# 135500
10
49.399126001456665 t# 135650
10
49.45375091041515 t# 135800
10
49.50837581937363 t# 135950
10
49.56300072833212 t# 136100
10
49.6176256372906 t# 136250
10
49.67225054624909 t# 136400
10
49.726875455207576 t# 136550
10
49.78150036416606 t# 136700
10
49.836125273124544 t# 136850
10
49.89075018208303 t# 137000
10
49.94537509104152 t# 137150
10
50.0 t# 137300
10
50.05462490895849 t# 137450
10
50.10924981791697 t# 137600
10
50.163874726875456 t# 137750
10
50.218499635833936 t# 137900
10
50.273124544792424 t# 138050
10
50.32774945375091 t# 138200
10
50.38237436270939 t# 138350
10
50.43699927166788 t# 138500
10
50.491624180626374 t# 138650
10
50.54624908

./data/Mix/sim4/s2.txt
[[174710 195780 'sim']
 [218260 241060 'sim']]
sim4  tMaxMin: 274650
1.820830298616169 t# 5000
1.8754552075746542 t# 5150
1.930080116533139 t# 5300
1.9847050254916243 t# 5450
2.0393299344501092 t# 5600
2.093954843408594 t# 5750
2.1485797523670795 t# 5900
2.2032046613255645 t# 6050
2.2578295702840494 t# 6200
2.3124544792425343 t# 6350
2.3670793882010197 t# 6500
2.421704297159505 t# 6650
2.4763292061179896 t# 6800
2.530954115076475 t# 6950
2.58557902403496 t# 7100
2.640203932993445 t# 7250
2.69482884195193 t# 7400
2.749453750910415 t# 7550
2.8040786598689005 t# 7700
2.858703568827385 t# 7850
2.9133284777858703 t# 8000
2.9679533867443557 t# 8150
3.02257829570284 t# 8300
3.0772032046613256 t# 8450
3.1318281136198105 t# 8600
3.186453022578296 t# 8750
3.241077931536781 t# 8900
3.295702840495266 t# 9050
3.3503277494537507 t# 9200
3.4049526584122356 t# 9350
3.459577567370721 t# 9500
3.5142024763292063 t# 9650
3.5688273852876913 t# 9800
3.6234522942461758 t# 9950
3.678077

10
63.98397669337218 t# 175700
10
64.03860160233066 t# 175850
10
64.09322651128915 t# 176000
10
64.14785142024763 t# 176150
10
64.20247632920612 t# 176300
10
64.25710123816461 t# 176450
10
64.31172614712308 t# 176600
10
64.36635105608157 t# 176750
10
64.42097596504006 t# 176900
10
64.47560087399854 t# 177050
10
64.53022578295703 t# 177200
10
64.58485069191552 t# 177350
10
64.639475600874 t# 177500
10
64.69410050983248 t# 177650
10
64.74872541879097 t# 177800
10
64.80335032774946 t# 177950
10
64.85797523670794 t# 178100
10
64.91260014566642 t# 178250
10
64.9672250546249 t# 178400
10
65.02184996358339 t# 178550
10
65.07647487254188 t# 178700
10
65.13109978150037 t# 178850
10
65.18572469045884 t# 179000
10
65.24034959941733 t# 179150
10
65.29497450837583 t# 179300
10
65.3495994173343 t# 179450
10
65.40422432629279 t# 179600
10
65.45884923525126 t# 179750
10
65.51347414420977 t# 179900
10
65.56809905316825 t# 180050
10
65.62272396212673 t# 180200
10
65.67734887108521 t# 180350
10
65.731973

10
79.77057538237436 t# 219050
10
79.82520029133285 t# 219200
10
79.87982520029134 t# 219350
10
79.93445010924982 t# 219500
10
79.9890750182083 t# 219650
10
80.04369992716678 t# 219800
10
80.09832483612527 t# 219950
10
80.15294974508376 t# 220100
10
80.20757465404225 t# 220250
10
80.26219956300072 t# 220400
10
80.31682447195921 t# 220550
10
80.37144938091771 t# 220700
10
80.42607428987618 t# 220850
10
80.48069919883467 t# 221000
10
80.53532410779314 t# 221150
10
80.58994901675163 t# 221300
10
80.64457392571013 t# 221450
10
80.6991988346686 t# 221600
10
80.7538237436271 t# 221750
10
80.80844865258557 t# 221900
10
80.86307356154407 t# 222050
10
80.91769847050256 t# 222200
10
80.97232337946103 t# 222350
10
81.02694828841952 t# 222500
10
81.081573197378 t# 222650
10
81.1361981063365 t# 222800
10
81.19082301529498 t# 222950
10
81.24544792425345 t# 223100
10
81.30007283321194 t# 223250
10
81.35469774217043 t# 223400
10
81.40932265112892 t# 223550
10
81.4639475600874 t# 223700
10
81.518572469

In [ ]:
unq,tags = np.unique(Y, return_inverse=True)
print(tags)
print(Y)
print(S)
print(np.unique(S))
Y=tags
print(np.unique(Y))



In [ ]:
def prinUserSensorMatrix(S,L,Y):
    df = pd.DataFrame()
    ts=[]
    ls=[]
    
    for t in range(0,X.shape[0]):
        ts.append(t)
    df['Time']=ts
    #print(ts)
    for s in range(0,X.shape[2]):
        ls.append("sensor-"+str(s))
        vs=S[:,:,s]
        vs=vs[:,0]
        print(vs.shape)
        df["sensor-"+str(s)]=vs
    

    
    ss=[]
    for t in range(0,X.shape[0]):
        ss.append(L[t])
    df['Scene']=ss
    ls.append('Scene')
    
    ys=[]
    for t in range(0,X.shape[0]):
        ys.append(Y[t])
    df['Label']=ys
    ls.append('Label')

    print(ls)
    
    df.plot(x="Time",y=ls,subplots=True, legend=True, figsize=(15,15))    
    
prinUserSensorMatrix(X,S,Y)

In [ ]:
print(X)       
print(X.shape)       

In [ ]:
Y=to_categorical(Y)

In [ ]:
print("nan:",np.argwhere(np.isnan(X)))
print(X[np.argwhere(np.isnan(X))])

In [10]:
from tensorflow import keras

from tensorflow.keras import layers
from keras.optimizers import Adam
from keras.layers import MaxPooling2D,Activation, Dense

from tensorflow import Tensor
from tensorflow.keras.layers import Input, Conv1D, Conv2D, ReLU, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense,Dropout,LSTM,TimeDistributed,MaxPooling1D
from tensorflow.keras.models import Model

from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt



classes=["der","sim","izq"]

Y_pred_total=np.array([])
Y_val_total=np.array([])

print(Y_val_total)


for sceneid in np.unique(S):

    print("SCENE:",sceneid)
    index_test=np.where(S == sceneid)[0]
    index_train=np.where(S != sceneid)[0]

    print("index_test",index_test)
    print("index_train",index_train)

    x_train=X[index_train]
    x_val=X[index_test]
    y_train=Y[index_train]
    y_val=Y[index_test]
    
    print(x_train)
    print(x_val)

    #normalizamos valores 
    for ixv in range(0,X.shape[2]):
        print(x_train[:,:,ixv])
        avg_v=np.nanmean(x_train[:,:,ixv].flatten())
        std_v=np.nanstd(x_train[:,:,ixv].flatten())
        print("avg_v,std_v:",avg_v,std_v)
        x_train[:,:,ixv]=(x_train[:,:,ixv]-avg_v)/(std_v)
        x_val[:,:,ixv]=(x_val[:,:,ixv]-avg_v)/(std_v)

    model = keras.Sequential()

    model.add(Conv1D(filters=16, kernel_size=2, activation='relu', input_shape=(X.shape[1],X.shape[2])))
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(Dropout(0.25))

    model.add(LSTM(256, return_sequences=True))
    model.add(Dropout(0.25))
    model.add(LSTM(256))

    model.add(Flatten())
    model.add(Dense(2048))
    model.add(Dense(1024))
    model.add(Dense(len(classes), activation="softmax"))

    model.compile(loss='categorical_crossentropy', optimizer='rmsprop',  metrics=['accuracy'])
    model.summary()

    history = model.fit(x_train , y_train,
                        batch_size=64,
                validation_data=(x_val,y_val),
                epochs=50)


    y_pred = model.predict(x_val).argmax(axis=1)
    y_val=y_val.argmax(axis=1)
    print(y_pred)
    print(y_val)
    Y_pred_total=np.concatenate((Y_pred_total,y_pred))
    Y_val_total=np.concatenate((Y_val_total,y_val))

    

    

46/46 ━━━━━━━━━━━━━━━━━━━━ 12s 270ms/step - accuracy: 0.9604 - loss: 0.1531 - val_accuracy: 0.9735 - val_loss: 0.1276
Epoch 12/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 12s 261ms/step - accuracy: 0.9504 - loss: 0.2115 - val_accuracy: 0.9711 - val_loss: 0.1840
Epoch 13/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 12s 263ms/step - accuracy: 0.9758 - loss: 0.1224 - val_accuracy: 0.9653 - val_loss: 0.1330
Epoch 14/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 7s 154ms/step - accuracy: 0.9898 - loss: 0.0555 - val_accuracy: 0.9780 - val_loss: 0.1371
Epoch 15/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 8s 61ms/step - accuracy: 0.9890 - loss: 0.0534 - val_accuracy: 0.9677 - val_loss: 0.1748
Epoch 16/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 4s 94ms/step - accuracy: 0.9904 - loss: 0.0494 - val_accuracy: 0.9745 - val_loss: 0.2071
Epoch 17/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 4s 86ms/step - accuracy: 0.9752 - loss: 0.1470 - val_accuracy: 0.9733 - val_loss: 0.1544
Epoch 18/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 4s 86ms/step - accuracy: 0.9997 - loss: 0.0018 - val_accuracy: 0.9729 

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_4 (Conv1D)               │ (None, 100, 16)        │           720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 98, 32)         │         1,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 98, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_6 (Conv1D)               │ (None, 96, 64)         │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_7 (Conv1D)               │ (None, 94, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 94, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 94, 256)        │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 94, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 256)            │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2048)           │       526,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1024)           │     2,098,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │         3,075 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,580,339 (13.66 MB)

 Trainable params: 3,580,339 (13.66 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 7s 59ms/step - accuracy: 0.3984 - loss: 1.1673 - val_accuracy: 0.5374 - val_loss: 0.9989
Epoch 2/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - accuracy: 0.6064 - loss: 0.8943 - val_accuracy: 0.9314 - val_loss: 0.2518
Epoch 3/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - accuracy: 0.8490 - loss: 0.4452 - val_accuracy: 0.7130 - val_loss: 0.5725
Epoch 4/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 4s 56ms/step - accuracy: 0.9085 - loss: 0.2600 - val_accuracy: 0.9685 - val_loss: 0.1373
Epoch 5/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.9772 - loss: 0.1054 - val_accuracy: 0.9754 - val_loss: 0.0917
Epoch 6/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.9696 - loss: 0.1074 - val_accuracy: 0.9799 - val_loss: 0.0993
Epoch 7/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.9831 - loss: 0.0771 - val_accuracy: 0.9868 - val_loss: 0.0645
Epoch 8/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.9709 - loss: 0.1173 - val_accuracy: 0.9837 - v

In [11]:
print(Y_pred_total)
print(Y_val_total)

cm = confusion_matrix(Y_val_total, Y_pred_total) 
print(cm)
print('Classification Report TOTAL')
print(classification_report(Y_val_total, Y_pred_total, target_names=classes))

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)

disp.plot(cmap=plt.cm.Blues)
plt.show()

[2. 2. 2. ... 2. 2. 2.]
[0. 0. 0. ... 2. 2. 2.]
[[2874    3   56]
 [   3 2048   55]
 [  16   21 2762]]
Classification Report TOTAL
              precision    recall  f1-score   support

         der       0.99      0.98      0.99      2933
         sim       0.99      0.97      0.98      2106
         izq       0.96      0.99      0.97      2799

    accuracy                           0.98      7838
   macro avg       0.98      0.98      0.98      7838
weighted avg       0.98      0.98      0.98      7838

